In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split

In [18]:
# build spark session
spark = SparkSession.builder.appName("csv_change_timestamp").getOrCreate()

In [19]:
# create rdd
data_dir = "/Users/kimdohoon/git/datas/ml-latest/ratings.csv"
rdd = spark.sparkContext.textFile(data_dir).map(lambda line: line.split(',')) \
                                           .filter(lambda x: all(item != "" for item in x)) \
                                           .filter(lambda x: len(x) == 4)

In [20]:
def is_int_convertible(value):
    try:
        int(value)
        return True
    except ValueError:
        return False

# rdd.filter()
# 함수의 출력이 "논리깂"을 반환하도록 구성
filtered_rdd = rdd.filter(lambda x: is_int_convertible(x[3]))

In [21]:
def convert_timestamp(row):
    from datetime import datetime
    
    timestamp = int(row[3])
    dt_object = datetime.fromtimestamp(timestamp)
    formatted_date = dt_object.strftime('%Y-%m-%d')
    formatted_time = dt_object.strftime('%H:%M:%S')
    return (row[0], row[1], row[2], formatted_date, formatted_time)  # Assuming you have other columns in the CSV

# rdd.map()
# 함수의 출력이 "tuple"을 반환하도록 구성
converted_rdd = filtered_rdd.map(convert_timestamp)

In [22]:
extracted_rdd = converted_rdd.map(lambda row: (row[2], 1)) \
                             .reduceByKey(lambda x, y: x + y)

In [ ]:
# change rdd to dataframe
column = ['rating','count']
df = extracted_rdd.toDF(column)
df.show()